In [20]:
import numpy as np ;
import requests ;
import os ;


class LinClass(object):

  ## allocate weight and bias arrays and store ref 2 train data/labels
  def __init__(self, n_in, n_out, X, T):
    self.W = np.ones([n_in,n_out]) * 0.1 ;
    self.b = np.ones([1,n_out])* 0.1 ;
    self.X = X ;
    self.T = T ;
    self.N = X.shape[0] ;

  # nomal cross-entropy loss. Fill in your code here!
  def loss(self, Y, T): 
    return - (np.log(Y) * T).sum(axis=1).mean() ;

  def dLdb(self,Y,T):
    return -(np.mean(T-Y, axis=0))

  # fill in your code here!
  def dLdW(self, X, Y, T):
    return -(np.mean(np.outer(T-Y, X), axis=0))
    
  # softmax: fill in your code here!
  def S(self,X):   # X is N,d
    e = np.exp(X) ;  # N,d
    row_sums = e.sum(axis = 1) ;   # (N, )
    row_sums_bc = row_sums.reshape(-1,1) ; # N,1
    #row_sums_bc = row_sums.reshape(2,1) ;  # less elegant bec. requires knowledge of rows in X
    
    return e / row_sums_bc;  # N,d / N,1 = N,d
       
  # dummy model, fill in your code!
  def f(self,X):
    return self.S (np.matmul(X, self.W) + self.b) ;

  # performs a single gradient descent step
  # works with any size of X and T
  def train_step(self, X, T, eps):
    Y = self.f(X) ;
    loss = self.loss(Y,T) ;
    dLdb = self.dLdb(Y,T) ;      
    dLdW = self.dLdW(X, Y, T) ;  
    self.b -= eps * dLdb ;       ## b(i+1) = b(i) - eps * gradL
    self.W -= eps * dLdW ;       ## same
    return loss ;

  # perform multiple gradient descent steps and display loss. Does it go down??
  def train(self,max_it,eps):
    for it in range(0,max_it):
      print ("iut=", it, "loss=", self.train_step(self.X, self.T, eps)) ;

## read it into 
data = np.load("mnist.npz")
traind = data["arr_0"] ;
trainl = data["arr_2"] ;
traind = traind.reshape(60000,784)

# your code from here!!
lc = LinClass(784,10,traind,trainl) ;
x = np.array([[-1,-1,5],[1,1,2.]]) ;
y = lc.S(x) ;
print("Softmax is :\n", lc.S(x)) ;
print ("\nChecking S...:\n", y.sum(axis=1))

print ("\nf is :\n", lc.f(traind[0,:])) ;
# small value added because log(0) will give a MathError
Y_test = np.array([[0,0,0,0,0.5, 0,0.3, 0, 0.2,0,0]])+0.00000000001 ; 
T_test = np.array([[0,0,0,0,1. , 0,0  , 0, 0  ,0,0]])
print("\nCross-Entropy:", lc.loss(Y_test, T_test)) ;

print("\ndLdB :", lc.dLdb(Y_test, T_test))

print("\ndLdW :", lc.dLdW(x, Y_test, T_test))





Softmax is :
 [[0.00246652 0.00246652 0.99506695]
 [0.21194156 0.21194156 0.57611688]]

Checking S...:
 [1. 1.]

f is :
 [[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]]

Cross-Entropy: 0.6931471805399453

dLdB : [ 1.e-11  1.e-11  1.e-11  1.e-11 -5.e-01  1.e-11  3.e-01  1.e-11  2.e-01
  1.e-11  1.e-11]

dLdW : [-1.00000007e-11 -1.00000007e-11  5.00000034e-11  1.00000007e-11
  1.00000007e-11  2.00000014e-11]
